In [1]:
# Copyright (c) 2019-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

In [2]:
#
# Code to generate sentence representations from a pretrained model.
# This can be used to initialize a cross-lingual classifier, for instance.
#

In [1]:
import os
import torch

from src.utils import AttrDict
from src.data.dictionary import Dictionary, BOS_WORD, EOS_WORD, PAD_WORD, UNK_WORD, MASK_WORD
from src.model.transformer import TransformerModel

## Reload a pretrained model

In [2]:
model_path = '/home/nanda/XLM_modif/new_dumped/xlm_all_new_data/1/best-valid_mix_mlm_ppl.pth'
reloaded = torch.load(model_path)
params = AttrDict(reloaded['params'])
print("Supported languages: %s" % ", ".join(params.lang2id.keys()))

Supported languages: mix, ms, zh, en, other


## Build dictionary / update parameters / build model

In [3]:
# build dictionary / update parameters
dico = Dictionary(reloaded['dico_id2word'], reloaded['dico_word2id'], reloaded['dico_counts'])
params.n_words = len(dico)
params.bos_index = dico.index(BOS_WORD)
params.eos_index = dico.index(EOS_WORD)
params.pad_index = dico.index(PAD_WORD)
params.unk_index = dico.index(UNK_WORD)
params.mask_index = dico.index(MASK_WORD)

# build model / reload weights
model = TransformerModel(params, dico, True, True)
model.eval()
model.load_state_dict(reloaded['model'])


## Get sentence representations

Sentences have to be in the BPE format, i.e. tokenized sentences on which you applied fastBPE.

Below you can see an example for English, French, Spanish, German, Arabic and Chinese sentences.

In [4]:
# Below is one way to bpe-ize sentences
codes = "" # path to the codes of the model
fastbpe = os.path.join(os.getcwd(), 'tools/fastBPE/fast')

def to_bpe(sentences):
    # write sentences to tmp file
    with open('/tmp/sentences.bpe', 'w') as fwrite:
        for sent in sentences:
            fwrite.write(sent + '\n')
    
    # apply bpe to tmp file
    os.system('%s applybpe /tmp/sentences.bpe /tmp/sentences %s' % (fastbpe, codes))
    
    # load bpe-ized sentences
    sentences_bpe = []
    with open('/tmp/sentences.bpe') as f:
        for line in f:
            sentences_bpe.append(line.rstrip())
    
    return sentences_bpe


In [ ]:
Hal ini membawa 人@@ 民@@ 起@@ 義 under the opposition candidate Viktor Yus@@ h@@ chenko yang kemudian didapati dira@@ cun

In [10]:
# Below are already BPE-ized sentences

# list of (sentences, lang)
sentences = [
    'Hal ini membawa 人民起義 under the opposition candidate Viktor Yushchenko yang kemudian didapati diracun'
]
# bpe-ize sentences
sentences = to_bpe(sentences)
print('\n\n'.join(sentences))

# check how many tokens are OOV
n_w = len([w for w in ' '.join(sentences).split()])
n_oov = len([w for w in ' '.join(sentences).split() if w not in dico.word2id])
print('Number of out-of-vocab words: %s/%s' % (n_oov, n_w))

# add </s> sentence delimiters
# sentences = [(('</s> %s </s>' % sent.strip()).split()) for sent in sentences]
print(sentences)

Hal ini membawa 人民起義 under the opposition candidate Viktor Yushchenko yang kemudian didapati diracun
Number of out-of-vocab words: 3/14
['Hal ini membawa 人民起義 under the opposition candidate Viktor Yushchenko yang kemudian didapati diracun']


### Create batch

In [8]:
bs = len(sentences)
slen = max([len(sent) for sent in sentences])

word_ids = torch.LongTensor(slen, bs).fill_(params.pad_index)
for i in range(len(sentences)):
    sent = torch.LongTensor([dico.index(w) for w in sentences[i]])
    word_ids[:len(sent), i] = sent

lengths = torch.LongTensor([len(sent) for sent in sentences])
                             
# NOTE: No more language id (removed it in a later version)
# langs = torch.LongTensor([params.lang2id[lang] for _, lang in sentences]).unsqueeze(0).expand(slen, bs) if params.n_langs > 1 else None
langs = None


### Forward

In [9]:
tensor = model('fwd', x=word_ids, lengths=lengths, langs=langs, causal=False).contiguous()
print(tensor.size())

torch.Size([38, 6, 1280])


The variable `tensor` is of shape `(sequence_length, batch_size, model_dimension)`.

`tensor[0]` is a tensor of shape `(batch_size, model_dimension)` that corresponds to the first hidden state of the last layer of each sentence.

This is this vector that we use to finetune on the GLUE and XNLI tasks.